## 1. Semantic Kernel Introduction

### Overview of Semantic Kernel (SK) and Its Importance

**Semantic Kernel** is an open-source SDK from Microsoft that acts as middleware between your application code and AI large language models (LLMs). It enables developers to easily integrate AI into apps by letting AI agents call code functions and by orchestrating complex tasks. SK is *lightweight* and *modular*, designed for **enterprise-grade solutions** with features like telemetry and filters for responsible AI. Major companies (including Microsoft) leverage SK because it’s flexible and **future-proof** – you can swap in new AI models as they emerge without rewriting your code. In short, SK helps build **robust, scalable AI applications** that can evolve with advancing AI capabilities.

Key reasons why Semantic Kernel is important for AI application development:

- **Bridging AI and Code**: SK combines natural language **prompts** with your **existing code and APIs**, allowing AI to take actions. The AI can request a function call and SK will execute that function and return results back to the model. This bridges the gap between what the AI *intends* and what your code can do.
- **Plugins (Skills)**: You can expose functionalities (from simple math to complex business logic or external APIs) as SK **plugins**. By describing your code to the AI (via function definitions), the model can invoke these functions to fulfill user requests. This plugin architecture makes your AI solutions **modular and extensible**.
- **Enterprise-ready**: SK includes support for **security, observability, and compliance** (e.g. integration with Azure services, monitoring, content filtering). Hooks and filters ensure you can enforce policies (for instance, prevent sensitive data leakage).
- **Multi-modal & Future-Proof**: SK natively supports multiple AI services (OpenAI, Azure OpenAI, HuggingFace, etc.) and modalities. Chat-based APIs can be extended to voice or other modes. As new models (like vision-enabled models or better language models) come out, SK lets you plug them in without major changes.
- **Rapid Development**: By handling the heavy lifting of prompt orchestration, function calling, and memory management, SK enables faster development of AI features. You focus on defining *what* you want the AI to do (skills, prompts) and SK handles *how* to do it. Microsoft claims that SK helps “deliver AI solutions faster than any other SDK” due to its ability to **automatically call functions**.

---

### Services and Core Components of SK

Semantic Kernel's architecture revolves around a few core components and services:

- **Kernel**: The central object that orchestrates everything. The `Kernel` holds configuration for AI services, manages plugins (skills), coordinates function calls, and maintains contextual state (memory). You typically create one Kernel instance in your app and use it to register functions and perform AI queries.
- **AI Services**: SK connects to AI models for different tasks:
  - *Chat Models*: e.g. Azure OpenAI GPT-4o-mini or GPT-4o for natural language generation and understanding.
  - *Embedding Models*: for converting text to vector embeddings (used in memory/search).
  - *Other Modalities*: connectors for images, speech, etc., if needed.
  
Semantic Kernel can automatically read your `.env` (automatically read from root of project) to access Azure OpenAI using the following variables:

```
AZURE_OPENAI_ENDPOINT - The endpoint it should talk to by default
AZURE_OPENAI_API_KEY - The API Key it should use
AZURE_OPENAI_API_VERSION - Inference API version it should use per default
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME - Model deployment name it should use per default
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME - Embedding deployment name it should use per default
```

Before you continue, make sure your `.env` (copied from `.env.example`) is filled out correctly.

You configure the Kernel with the endpoints/keys for the services you need. For example, adding an Azure OpenAI chat completion service:

---

In [ ]:
!pip install semantic-kernel==1.29.0 python-dotenv==1.0.1

Before continuing, please quickly restart your Jupyter kernel, so that SK can load properly.

In [1]:
import os
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()

kernel.add_service(chat_completion)

You have now successfully created a kernel and added a chat completion service.

Similarly, you can add an embedding generation service via `kernel.add_service(text_embedding)` if performing semantic memory searches. But don't worry we will dive into this at a later stage.

---

### Functions and Plugins in SK

**Functions** in Semantic Kernel are the actions that the AI can perform. They come in two types:

- **Semantic Functions**: Backed by a prompt and LLM. For example, a function `translateToFrench` might use the prompt `"Translate this to French: {{$input}}"`.
- **Native Functions**: Backed by code. For example, a Python function `sendEmail(to, subject, body)` that uses an API to send an email.

These functions are typically grouped into **Plugins** (or "Skills"). Those are often represented as a class in Python. Grouping functions into plugins helps manage and control which capabilities are exposed to the AI.

**Using Plugins/Functions**: Once registered with the kernel (via `kernel.add_function` or `kernel.add_plugin`), functions become available for invocation. They can be called directly in code via `kernel.invoke(function, input)`, or the AI model can automatically choose to invoke them as needed.

SK’s plugin system is highly flexible:
- You can load **OpenAPI** specifications or API endpoints as plugins.
- Plugins can be shared across projects, allowing organizations to build a library of useful AI plugins.

---

You can add plugins to the Kernel in various ways:

- **Inline Definition**: Define a prompt or function in code and register it.
- **From Files or Classes**: Load plugins from directories or Python classes decorated appropriately.

For example, to add a simple **semantic function** inline:

In [2]:
# Define a semantic function (prompt) to generate a TL;DR summary
prompt_template = "{{$input}}\n\nTL;DR in one sentence:"

summarize_fn = kernel.add_function(
    prompt=prompt_template,
    function_name="tldr",
    plugin_name="Summarizer",
    max_tokens=100,
)

# Use the function
long_text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your C#, 
Python, or Java codebase. It serves as an efficient middleware that enables 
rapid delivery of enterprise-grade solutions.
"""

summary = await kernel.invoke(summarize_fn, input=long_text)
print(summary)

Semantic Kernel is an open-source development kit for building AI agents and integrating advanced AI models into C#, Python, or Java applications.


### Exercise: Create a translation function

Write a semantic function that translates text to a target language.

<details>
  <summary>Click to see solution</summary>
  
  ```python
  # Define a semantic function (prompt) to generate a translation
prompt_template = "{{$input}}\n\nTranslate this into {{$target_lang}}:"

translate_fn = kernel.add_function(
    prompt=prompt_template, 
    function_name="translator", 
    plugin_name="Translator",
    max_tokens=50)

# Use the function
text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your C#, 
Python, or Java codebase. It serves as an efficient middleware that enables 
rapid delivery of enterprise-grade solutions.
"""

summary = await kernel.invoke(translate_fn, input=text, target_lang="French")
print(summary)
  ```
</details>

In [5]:
# Take a look at the previous implementation for reference, if you are stuck view the solution.

# Your solution goes here
# Define a semantic function (prompt) to translate text
prompt = "Translate this text to {{$target_language}} using a simple and non-formal conversational style: {{$input_text}}"

translate_fn = kernel.add_function(
    prompt=prompt,
    function_name="translate_to_japanese",
    plugin_name="Translator",
    max_tokens=1000,
)

# Use the function
long_text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your C#, 
Python, or Java codebase. It serves as an efficient middleware that enables 
rapid delivery of enterprise-grade solutions.
"""

translation = await kernel.invoke(translate_fn, target_language="japanese", input_text=long_text)
print(translation)

セマンティックカーネルは、軽量でオープンソースの開発キットだよ。これを使えば、AIエージェントを簡単に作ったり、最新のAIモデルをC#、Python、Javaのコードに簡単に組み込めるんだ。効率的なミドルウェアとして動くから、エンタープライズ向けのソリューションを素早く作れるよ。



Now if we want to build a Plugin with a set of **native functions** we can do the following:

In [7]:
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions import kernel_function


class LightModel(TypedDict):
    id: int
    name: str
    is_on: bool | None
    brightness: int | None
    hex: str | None


class LightsPlugin:
    def __init__(self, lights: list[LightModel]):
        self.lights = lights

    @kernel_function
    async def get_lights(self) -> List[LightModel]:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function
    async def get_state(
        self, id: Annotated[int, "The ID of the light"]
    ) -> Optional[LightModel]:
        """Gets the state of a particular light."""
        for light in self.lights:
            if light["id"] == id:
                return light
        return None

    @kernel_function
    async def change_state(
        self, id: Annotated[int, "The ID of the light"], new_state: LightModel
    ) -> Optional[LightModel]:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = new_state.get("is_on", light["is_on"])
                light["brightness"] = new_state.get("brightness", light["brightness"])
                light["hex"] = new_state.get("hex", light["hex"])
                return light
        return None

In [8]:
# Create dependencies for the plugin
lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

plugin = LightsPlugin(lights=lights)

kernel.add_plugin(
    plugin=plugin,
    plugin_name="Lights",
)

KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light.', parameters=[KernelParameterMetadata(name='id', description='The ID of the light', default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer', 'description': 'The ID of the light'}, include_in_function_choices=True), KernelParameterMetadata(name='new_state', description=None, default_value=None, type_='LightModel', is_required=True, type_object=<class '__main__.LightModel'>, schema_data={'type': 'object', 'properties': {'id': {'type': 'integer'}, 'name': {'type': 'string'}, 'is_on': {'type': ['boolean', 'null']}, 'brightness': {'type': ['integer', 'null']}, 'hex': {'type': ['string', 'null']}}, 'required': ['id', 'name']}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelPa

In [9]:
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
history.add_user_message("Please turn on the lamp")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Print the results
print("Assistant > " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)

Assistant > The Table Lamp has been turned on.


### Automatic Function Calling

One of the most powerful features of Semantic Kernel is its ability to **automatically orchestrate multi-step operations** by calling multiple functions in sequence. In this example, a **LightsPlugin** is defined with three asynchronous native functions:

- **get_lights()**: Retrieves the list of lights along with their current states.
- **get_state(id)**: Returns the state of a specific light, given its ID.
- **change_state(id, new_state)**: Changes the state of a specified light to a new state (for example, turning it on, adjusting brightness, or changing its color).

**How It Works**:
- The kernel exposes all registered functions to the AI model.
- When a user issues a command like *"Turn on all the lights and give me their final state,"* the AI model analyzes the request and plans the necessary steps:
  1. Call **get_lights()** to retrieve all available lights.
  2. For each light, invoke **change_state()** with a new state (e.g., setting `"is_on": True`).
  3. Optionally, call **get_state()** for each light to confirm the updated status.
- The kernel then returns a comprehensive result that reflects the final state of each light.

**Example Scenario**:
- **User Query**: *"Turn on all the lights and tell me their status."*
- **Step 1**: The system calls `LightsPlugin.get_lights()` to fetch the current list of lights.
- **Step 2**: It iterates over the list and calls `LightsPlugin.change_state(id, new_state)` to turn each light on.
- **Step 3**: Finally, it may call `LightsPlugin.get_state(id)` for each light to confirm the changes.
- The final output, including the updated state for each light, is returned to the user.

This automatic orchestration simplifies the management of multi-step tasks, enabling the AI to autonomously plan and execute function calls without manual intervention.

In [14]:
## Try other messages and observe the results

# Add the message from the user to the chat history
# history.add_user_message("Please turn on all the lamps")
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
history.add_user_message("Turn off all the lamps, tell me their final state, then turn them all on, and tell me their final state, \
    then turn off the table lamp and tell me the final state of all their lamps. ")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Print the results
print("Assistant > " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)

# history.add_user_message("Please turn off all the lamps and give me their final state")

Assistant > ### Final states of all lamps at various steps:

1. **After turning off all lamps:**
   - **Table Lamp:** Off, Brightness: 100, Color HEX: FF0000
   - **Porch Light:** Off, Brightness: 50, Color HEX: 00FF00
   - **Chandelier:** Off, Brightness: 75, Color HEX: 0000FF

2. **After turning on all lamps:**
   - **Table Lamp:** On, Brightness: 100, Color HEX: FF0000
   - **Porch Light:** On, Brightness: 50, Color HEX: 00FF00
   - **Chandelier:** On, Brightness: 75, Color HEX: 0000FF

3. **After turning off the Table Lamp while others are still on:**
   - **Table Lamp:** Off, Brightness: 100, Color HEX: FF0000
   - **Porch Light:** On, Brightness: 50, Color HEX: 00FF00
   - **Chandelier:** On, Brightness: 75, Color HEX: 0000FF


----

# Exercise: Creating a Weather Information Plugin

In this exercise, you'll create a native plugin for Semantic Kernel that simulates a weather service. This will help you understand how to create and use native functions in a plugin.

### Task:
1. Create a `WeatherPlugin` class with the following functions:
   - `get_current_weather(location)`: Returns the current weather for a given location
   - `get_forecast(location, days)`: Returns a weather forecast for a specified number of days
   - `get_weather_alert(location)`: Returns any active weather alerts for a location

2. Register the plugin with the kernel and test it with a few user queries


<details>
  <summary>Click to see solution</summary>
  
  ```python
    from typing import Annotated, List, Dict
    from semantic_kernel.functions import kernel_function
    import random

    class WeatherPlugin:
        def __init__(self):
            # Simulated weather data
            self.weather_conditions = ["Sunny", "Cloudy", "Rainy", "Snowy", "Windy", "Foggy", "Stormy"]
            self.temperature_ranges = {
                "New York": (50, 85),
                "London": (45, 75),
                "Tokyo": (55, 90),
                "Sydney": (60, 95),
                "Paris": (48, 80),
                "Default": (40, 100)
            }
            
            # Simulated alerts
            self.alerts = {
                "New York": "Heat advisory in effect",
                "Tokyo": "Typhoon warning for coastal areas",
                "Sydney": None,
                "London": None,
                "Paris": "Air quality warning"
            }
        
        @kernel_function
        async def get_current_weather(
            self,
            location: Annotated[str, "The city name to get weather for"]
        ) -> Dict:
            """Gets the current weather for a specified location."""
            temp_range = self.temperature_ranges.get(location, self.temperature_ranges["Default"])
            temperature = random.randint(temp_range[0], temp_range[1])
            condition = random.choice(self.weather_conditions)
            
            return {
                "location": location,
                "temperature": temperature,
                "condition": condition,
                "humidity": random.randint(30, 95),
                "wind_speed": random.randint(0, 30)
            }
        
        @kernel_function
        async def get_forecast(
            self,
            location: Annotated[str, "The city name to get forecast for"],
            days: Annotated[int, "Number of days for the forecast"] = 3
        ) -> List[Dict]:
            """Gets a weather forecast for a specified number of days."""
            forecast = []
            temp_range = self.temperature_ranges.get(location, self.temperature_ranges["Default"])
            
            for i in range(days):
                forecast.append({
                    "day": i + 1,
                    "temperature": random.randint(temp_range[0], temp_range[1]),
                    "condition": random.choice(self.weather_conditions),
                    "humidity": random.randint(30, 95),
                    "wind_speed": random.randint(0, 30)
                })
            
            return forecast
        
        @kernel_function
        async def get_weather_alert(
            self,
            location: Annotated[str, "The city name to check for weather alerts"]
        ) -> Dict:
            """Gets any active weather alerts for a location."""
            alert = self.alerts.get(location)
            
            return {
                "location": location,
                "has_alert": alert is not None,
                "alert_message": alert if alert else "No active alerts"
            }

    # Usage example:
    # Create the plugin
    weather_plugin = WeatherPlugin()

    # Register with kernel
    kernel.add_plugin(
        plugin=weather_plugin,
        plugin_name="Weather"
    )

    # Test with a user query
    history = ChatHistory()
    history.add_user_message("What's the weather like in Tokyo and are there any alerts?")

    # Get response using function calling
    execution_settings = AzureChatPromptExecutionSettings()
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    print("Assistant > " + str(result))
```
</details>


----

### Learning Objectives:
- Creating a native plugin with multiple functions
- Using type annotations for function parameters
- Registering a plugin with the kernel
- Testing the plugin with natural language queries


In [16]:
## TODO: Exercise

from typing import Annotated, List, Dict
from semantic_kernel.functions import kernel_function
import random

class WeatherPlugin:
    def __init__(self):
        # Simulated weather data
        self.weather_conditions = ["Sunny", "Cloudy", "Rainy", "Snowy", "Windy", "Foggy", "Stormy"]
        self.temperature_ranges = {
            "New York": (50, 85),
            "London": (45, 75),
            "Tokyo": (55, 90),
            "Sydney": (60, 95),
            "Paris": (48, 80),
            "Default": (40, 100)
        }

        # Simulated alerts
        self.alerts = {
            "New York": "Heat advisory in effect",
            "Tokyo": "Typhoon warning for coastal areas",
            "Sydney": None,
            "London": None,
            "Paris": "Air quality warning"
        }

    @kernel_function
    async def get_current_weather(
        self,
        location: Annotated[str, "The city name to get weather for"]
    ) -> Dict:
        """Gets the current weather for a specified location."""
        temp_range = self.temperature_ranges.get(location, self.temperature_ranges["Default"])
        temperature = random.randint(temp_range[0], temp_range[1])
        condition = random.choice(self.weather_conditions)

        return {
            "location": location,
            "temperature": temperature,
            "condition": condition,
            "humidity": random.randint(30, 95),
            "wind_speed": random.randint(0, 30)
        }

    @kernel_function
    async def get_forecast(
        self,
        location: Annotated[str, "The city name to get forecast for"],
        days: Annotated[int, "Number of days for the forecast"] = 3
    ) -> List[Dict]:
        """Gets a weather forecast for a specified number of days."""
        forecast = []
        temp_range = self.temperature_ranges.get(location, self.temperature_ranges["Default"])

        for i in range(days):
            forecast.append({
                "day": i + 1,
                "temperature": random.randint(temp_range[0], temp_range[1]),
                "condition": random.choice(self.weather_conditions),
                "humidity": random.randint(30, 95),
                "wind_speed": random.randint(0, 30)
            })

        return forecast

    @kernel_function
    async def get_weather_alert(
        self,
        location: Annotated[str, "The city name to check for weather alerts"]
    ) -> Dict:
        """Gets any active weather alerts for a location."""
        alert = self.alerts.get(location)

        return {
            "location": location,
            "has_alert": alert is not None,
            "alert_message": alert if alert else "No active alerts"
        }

# Usage example:
# Create the plugin
weather_plugin = WeatherPlugin()

# Register with kernel
kernel.add_plugin(
    plugin=weather_plugin,
    plugin_name="Weather"
)

# Test with a user query
history = ChatHistory()
history.add_user_message("What's the weather like in Tokyo and are there any alerts?")

# Get response using function calling
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

print("Assistant > " + str(result))

Assistant > The weather in Tokyo is currently 78°F with cloudy skies. The humidity is at 54%, and there's a wind speed of 28 mph.

Additionally, there is an active weather alert: **Typhoon warning for coastal areas**.


### Filters in Semantic Kernel

Filters provide a layer of control and visibility over function execution, ensuring responsible AI practices and enterprise-grade security. They allow you to:

- **Validate Permissions:**  
  For example, a filter can check user permissions before initiating an approval flow.

- **Intercept Function Execution:**  
  - **Function Invocation Filter:**  
    Runs every time a function is called; it can access function details, handle exceptions, override results (e.g., for caching or responsible AI), or retry on failure.
  - **Prompt Render Filter:**  
    Triggered before a prompt is rendered; it allows you to view or modify the prompt and even override the result to prevent submission.
  - **Auto Function Invocation Filter:**  
    Works within automatic function calling, providing additional context (like chat history and iteration counters) and can terminate the process early if needed.

Each filter receives a context object with execution details and must call the next delegate (or callback) to continue the execution chain. Filters can be registered either by using the `add_filter` method on the Kernel or via the `@kernel.filter` decorator.


One of the things that I would like to improve in our plugin implementation is to add debugging, that way I can integrate with external systems for auditing purposes

Lets implement that

In [17]:
import logging

# Configure the logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

In [18]:
from typing import Awaitable, Callable
from semantic_kernel.filters import FunctionInvocationContext

logger = logging.getLogger(__name__)


async def logger_filter(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    logger.info(f"FunctionInvoking - {context.function.plugin_name}.{context.function.name}")

    await next(context)

    logger.info(f"FunctionInvoked - {context.function.plugin_name}.{context.function.name}")


kernel.add_filter("function_invocation", logger_filter)

In [19]:
history.add_user_message("Please turn on all the lamps and give me their final state")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Add the message from the agent to the chat history
history.add_message(result)


# Print the results
print("Assistant > " + str(result))

2025-05-17 19:06:55,171 - INFO - HTTP Request: POST https://azopenaires.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-17 19:06:55,173 - INFO - OpenAI usage: CompletionUsage(completion_tokens=14, prompt_tokens=419, total_tokens=433, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-17 19:06:55,175 - INFO - processing 1 tool calls in parallel.
2025-05-17 19:06:55,176 - INFO - Calling Lights-get_lights function with args: {}
2025-05-17 19:06:55,177 - INFO - Function Lights-get_lights invoking.
2025-05-17 19:06:55,177 - INFO - FunctionInvoking - Lights.get_lights
2025-05-17 19:06:55,178 - INFO - FunctionInvoked - Lights.get_lights
2025-05-17 19:06:55,178 - INFO - Function Lights-get_lights succeeded.
2025-05-17 19:06:55,178 - INFO - Function 

Assistant > All lamps have been turned on. Here is their final state:

1. **Table Lamp**: On, Brightness: 100, Color: #FF0000 (Red)
2. **Porch light**: On, Brightness: 50, Color: #00FF00 (Green)
3. **Chandelier**: On, Brightness: 75, Color: #0000FF (Blue)


The output should look something like this

```shell
2025-02-27 15:21:00,969 - INFO - HTTP Request: POST <AOAI Endpoint> "HTTP/1.1 200 OK"
2025-02-27 15:21:00,971 - INFO - OpenAI usage: CompletionUsage(completion_tokens=84, prompt_tokens=407, total_tokens=491, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-02-27 15:21:00,971 - INFO - processing 2 tool calls in parallel.
2025-02-27 15:21:00,972 - INFO - Calling Lights-change_state function with args: {"id": 2, "new_state": {"id": 2, "name": "Porch light", "is_on": true}}
2025-02-27 15:21:00,972 - INFO - Function Lights-change_state invoking.
2025-02-27 15:21:00,972 - INFO - FunctionInvoking - Lights.change_state
2025-02-27 15:21:00,972 - INFO - FunctionInvoked - Lights.change_state
2025-02-27 15:21:00,973 - INFO - Function Lights-change_state succeeded.
2025-02-27 15:21:00,973 - INFO - Function completed. Duration: 0.000904s
2025-02-27 15:21:00,974 - INFO - Calling Lights-change_state function with args: {"id": 3, "new_state": {"id": 3, "name": "Chandelier", "is_on": true}}
2025-02-27 15:21:00,974 - INFO - Function Lights-change_state invoking.
2025-02-27 15:21:00,974 - INFO - FunctionInvoking - Lights.change_state
2025-02-27 15:21:00,974 - INFO - FunctionInvoked - Lights.change_state
2025-02-27 15:21:00,974 - INFO - Function Lights-change_state succeeded.
2025-02-27 15:21:00,975 - INFO - Function completed. Duration: 0.000567s
2025-02-27 15:21:06,333 - INFO - HTTP Request: POST <AOAI Endpoint> "HTTP/1.1 200 OK"
2025-02-27 15:21:06,335 - INFO - OpenAI usage: CompletionUsage(completion_tokens=120, prompt_tokens=572, total_tokens=692, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
Assistant > All the lights are now turned on. Here's the final state of each lamp:

1. **Table Lamp**
   - Status: On
   - Brightness: 100%
   - Color: Red (#FF0000)

2. **Porch Light**
   - Status: On
   - Brightness: 50%
   - Color: Green (#00FF00)

3. **Chandelier**
   - Status: On
   - Brightness: 75%
   - Color: Blue (#0000FF)

If you need any further adjustments, just let me know!
```

## Exercise: Creating a Content Filter

In this exercise, you'll create a filter that monitors and potentially modifies content flowing through the Semantic Kernel pipeline. This will help you understand how to implement safety and compliance measures in AI applications.

### Task:
1. Create a content filter that detects and redacts sensitive information (like credit card numbers, emails, etc.)
2. Implement the filter as a pre-processing step for user inputs and a post-processing step for AI outputs
3. Test the filter with various inputs containing sensitive information

<details>
  <summary>Click to see solution</summary>
  
  ```python
import re
from typing import Any, Coroutine
from collections.abc import Callable as ABCCallable
import logging
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.functions import FunctionResult

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Regular expressions for sensitive data patterns
PATTERNS = {
    'credit_card': r'\b(?:\d{4}[-\s]?){3}\d{4}\b',  # Credit card format: XXXX-XXXX-XXXX-XXXX
    'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email addresses
    'phone': r'\b(?:\+\d{1,3}[-\s]?)?\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{4}\b',  # Phone numbers
    'ssn': r'\b\d{3}[-\s]?\d{2}[-\s]?\d{4}\b',  # Social Security Numbers (US)
}

class ContentFilter:
    def __init__(self, patterns=PATTERNS):
        self.patterns = patterns
        
    def redact_sensitive_info(self, text):
        """Redact sensitive information from text."""
        result = text
        detected = []
        
        for pattern_name, pattern in self.patterns.items():
            matches = re.finditer(pattern, result)
            for match in matches:
                detected.append(f"{pattern_name}: {match.group()}")
                result = result.replace(match.group(), f"[REDACTED {pattern_name.upper()}]")
        
        return result, detected

# Create a pre-processing filter for user inputs
async def input_filter(
    context: FunctionInvocationContext,
    next: ABCCallable[[FunctionInvocationContext], Coroutine[Any, Any, None]]
) -> None:
    content_filter = ContentFilter()
    
    # Check if there's an input parameter
    if 'input' in context.arguments:
        original_input = context.arguments['input']
        
        # Apply the filter
        filtered_input, detected = content_filter.redact_sensitive_info(original_input)
        
        if detected:
            logger.warning(f"Sensitive information detected in input: {', '.join(detected)}")
            
        # Replace the original input with the filtered version
        context.arguments['input'] = filtered_input
    
    # Continue to the next filter or function
    await next(context)

# Create a post-processing filter for AI outputs
async def output_filter(
    context: FunctionInvocationContext,
    next: ABCCallable[[FunctionInvocationContext], Coroutine[Any, Any, None]]
) -> None:
    # Continue to the next filter or function first
    await next(context)
    
    content_filter = ContentFilter()
    
    # Check if there's a result to filter
    if context.result:
        original_output = str(context.result)
        
        # Apply the filter
        filtered_output, detected = content_filter.redact_sensitive_info(original_output)
        
        if detected:
            logger.warning(f"Sensitive information detected in output: {', '.join(detected)}")
         
        # Create a new FunctionResult with the filtered output
        context.result = FunctionResult(
            function=context.function.metadata,
            value=filtered_output,
            metadata=context.result.metadata if hasattr(context.result, 'metadata') else {}
        )

async def test_content_filters():
    
    # Register the filters with the kernel
    kernel.add_filter("function_invocation", input_filter)
    kernel.add_filter("function_invocation", output_filter)
    
    # Create a simple semantic function
    echo_prompt = "{{$input}}"
    echo_fn = kernel.add_function(
        prompt=echo_prompt,
        function_name="echo",
        plugin_name="TestPlugin"
    )
    
    # Test with sensitive information
    test_inputs = [
        "My credit card number is 4111-1111-1111-1111",
        "Contact me at john.doe@example.com or call 555-123-4567",
        "My SSN is 123-45-6789",
        "Here's my info: john.doe@example.com, 4111-1111-1111-1111, 555-123-4567",
        "What are the services that you can offer?"
    ]
    
    for input_text in test_inputs:
        print(f"\nOriginal Input: {input_text}")
        result = await kernel.invoke(echo_fn, input=input_text)
        print(f"Filtered Output: {result}")

await test_content_filters()
```
</details>




### Learning Objectives:
- Creating and registering filters in Semantic Kernel
- Implementing pre-processing and post-processing logic
- Using regular expressions for pattern matching
- Understanding the filter execution pipeline
- Logging and monitoring sensitive information

----

### Filters in SK and Their Use Cases

To summarize, in any AI application, it’s important to control input/output and function execution for security, privacy, and correctness. **Filters** in Semantic Kernel act as middleware or interceptors in the execution pipeline.

**Use Cases for Filters**:
- **Security/Policy**: Prevent sensitive data from being sent to the AI.
- **Validation**: Check function arguments before execution.
- **Error Handling**: Catch exceptions and provide default results.
- **Logging/Monitoring**: Log each function call and its response.
- **Post-processing**: Modify outputs before they’re returned to the AI.

In [28]:
import re
from typing import Any, Coroutine
from collections.abc import Callable as ABCCallable
import logging
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.functions import FunctionResult

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Regular expressions for sensitive data patterns
PATTERNS = {
    'credit_card': r'\b(?:\d{4}[-\s]?){3}\d{4}\b',  # Credit card format: XXXX-XXXX-XXXX-XXXX
    'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email addresses
    'phone': r'\b(?:\+\d{1,3}[-\s]?)?\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{4}\b',  # Phone numbers
    'ssn': r'\b\d{3}[-\s]?\d{2}[-\s]?\d{4}\b',  # Social Security Numbers (US)
}

class ContentFilter:
    def __init__(self, patterns=PATTERNS):
        self.patterns = patterns

    def redact_sensitive_info(self, text):
        """Redact sensitive information from text."""
        result = text
        detected = []

        for pattern_name, pattern in self.patterns.items():
            matches = re.finditer(pattern, result)
            for match in matches:
                detected.append(f"{pattern_name}: {match.group()}")
                result = result.replace(match.group(), f"[REDACTED {pattern_name.upper()}]")

        return result, detected

# Create a pre-processing filter for user inputs
async def input_filter(
    context: FunctionInvocationContext,
    next: ABCCallable[[FunctionInvocationContext], Coroutine[Any, Any, None]]
) -> None:
    content_filter = ContentFilter()

    # Check if there's an input parameter
    if 'input' in context.arguments:
        original_input = context.arguments['input']

        # Apply the filter
        filtered_input, detected = content_filter.redact_sensitive_info(original_input)
        print("filtered input!!!!!!: "+ filtered_input)
        if detected:
            logger.warning(f"Sensitive information detected in input!!!!!: {', '.join(detected)}")
            logger.warning(f"Sensitive information detected in input, filtered: {', '.join(filtered_input)}")

        # Replace the original input with the filtered version
        context.arguments['input'] = filtered_input

    # Continue to the next filter or function
    await next(context)

# Create a post-processing filter for AI outputs
async def output_filter(
    context: FunctionInvocationContext,
    next: ABCCallable[[FunctionInvocationContext], Coroutine[Any, Any, None]]
) -> None:
    # Continue to the next filter or function first
    await next(context)

    content_filter = ContentFilter()

    # Check if there's a result to filter
    if context.result:
        original_output = str(context.result)

        # Apply the filter
        filtered_output, detected = content_filter.redact_sensitive_info(original_output)

        if detected:
            logger.warning(f"Sensitive information detected in output: {', '.join(detected)}")
            logger.warning(f"Sensitive information detected in output, filtered: {', '.join(filtered_output)}")

        # Create a new FunctionResult with the filtered output
        context.result = FunctionResult(
            function=context.function.metadata,
            value=filtered_output,
            metadata=context.result.metadata if hasattr(context.result, 'metadata') else {}
        )

async def test_content_filters():

    # Register the filters with the kernel
    kernel.add_filter("function_invocation", input_filter)
    kernel.add_filter("function_invocation", output_filter)

    # Create a simple semantic function
    echo_prompt = "{{$input}}"
    echo_fn = kernel.add_function(
        prompt=echo_prompt,
        function_name="echo",
        plugin_name="TestPlugin"
    )

    # Test with sensitive information
    test_inputs = [
        "My credit card number is 4111-1111-1111-1111",
        "Contact me at john.doe@example.com or call 555-123-4567",
        "My SSN is 123-45-6789",
        "Here's my info: john.doe@example.com, 4111-1111-1111-1111, 555-123-4567",
        "What are the services that you can offer?"
    ]

    for input_text in test_inputs:
        print(f"\nOriginal Input: {input_text}")
        result = await kernel.invoke(echo_fn, input=input_text)
        print(f"Filtered Output: {result}")

await test_content_filters()

2025-05-17 19:31:34,096 - INFO - Function TestPlugin-echo invoking.
2025-05-17 19:31:34,097 - INFO - FunctionInvoking - TestPlugin.echo
2025-05-17 19:31:34,097 - WARNING - Sensitive information detected in input: credit_card: 4111-1111-1111-1111



Original Input: My credit card number is 4111-1111-1111-1111
filtered input!!!!!!: My credit card number is [REDACTED CREDIT_CARD]
filtered input!!!!!!: My credit card number is [REDACTED CREDIT_CARD]
filtered input!!!!!!: My credit card number is [REDACTED CREDIT_CARD]
filtered input!!!!!!: My credit card number is [REDACTED CREDIT_CARD]


2025-05-17 19:31:35,376 - INFO - HTTP Request: POST https://azopenaires.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-17 19:31:35,377 - INFO - OpenAI usage: CompletionUsage(completion_tokens=51, prompt_tokens=20, total_tokens=71, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-17 19:31:35,378 - INFO - FunctionInvoked - TestPlugin.echo
2025-05-17 19:31:35,378 - INFO - Function TestPlugin-echo succeeded.
2025-05-17 19:31:35,378 - INFO - Function completed. Duration: 1.281677s
2025-05-17 19:31:35,379 - INFO - Function TestPlugin-echo invoking.
2025-05-17 19:31:35,379 - INFO - FunctionInvoking - TestPlugin.echo
2025-05-17 19:31:35,380 - WARNING - Sensitive information detected in input: email: john.doe@example.com, phone: 555-123-4567


Filtered Output: I'm sorry, but I can't assist with that. Please do not share sensitive personal information, such as credit card numbers, online for your own safety and privacy. If you have a question or need help, feel free to ask without including private details!

Original Input: Contact me at john.doe@example.com or call 555-123-4567
filtered input!!!!!!: Contact me at [REDACTED EMAIL] or call [REDACTED PHONE]
filtered input!!!!!!: Contact me at [REDACTED EMAIL] or call [REDACTED PHONE]
filtered input!!!!!!: Contact me at [REDACTED EMAIL] or call [REDACTED PHONE]
filtered input!!!!!!: Contact me at [REDACTED EMAIL] or call [REDACTED PHONE]


2025-05-17 19:31:35,873 - INFO - HTTP Request: POST https://azopenaires.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-17 19:31:35,874 - INFO - OpenAI usage: CompletionUsage(completion_tokens=25, prompt_tokens=26, total_tokens=51, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-17 19:31:35,875 - INFO - FunctionInvoked - TestPlugin.echo
2025-05-17 19:31:35,875 - INFO - Function TestPlugin-echo succeeded.
2025-05-17 19:31:35,876 - INFO - Function completed. Duration: 0.496178s
2025-05-17 19:31:35,876 - INFO - Function TestPlugin-echo invoking.
2025-05-17 19:31:35,876 - INFO - FunctionInvoking - TestPlugin.echo
2025-05-17 19:31:35,877 - WARNING - Sensitive information detected in input: ssn: 123-45-6789


...

Filtered Output: I'm sorry, but I can't process or store sensitive personal information like Social Security Numbers (SSNs). Please make sure to keep your personal information private and secure. Let me know if there's anything else I can assist you with!

Original Input: Here's my info: john.doe@example.com, 4111-1111-1111-1111, 555-123-4567
filtered input!!!!!!: Here's my info: [REDACTED EMAIL], [REDACTED CREDIT_CARD], [REDACTED PHONE]
filtered input!!!!!!: Here's my info: [REDACTED EMAIL], [REDACTED CREDIT_CARD], [REDACTED PHONE]
filtered input!!!!!!: Here's my info: [REDACTED EMAIL], [REDACTED CREDIT_CARD], [REDACTED PHONE]
filtered input!!!!!!: Here's my info: [REDACTED EMAIL], [REDACTED CREDIT_CARD], [REDACTED PHONE]


2025-05-17 19:31:37,549 - INFO - HTTP Request: POST https://azopenaires.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-17 19:31:37,550 - INFO - OpenAI usage: CompletionUsage(completion_tokens=53, prompt_tokens=33, total_tokens=86, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-17 19:31:37,551 - INFO - FunctionInvoked - TestPlugin.echo
2025-05-17 19:31:37,551 - INFO - Function TestPlugin-echo succeeded.
2025-05-17 19:31:37,551 - INFO - Function completed. Duration: 0.891299s
2025-05-17 19:31:37,552 - INFO - Function TestPlugin-echo invoking.
2025-05-17 19:31:37,552 - INFO - FunctionInvoking - TestPlugin.echo


Filtered Output: I'm sorry, but I can't process or store sensitive personal information such as email addresses, credit card details, or phone numbers. For your privacy and security, please avoid sharing such information online. If you have a question or need assistance, feel free to ask!

Original Input: What are the services that you can offer?
filtered input!!!!!!: What are the services that you can offer?
filtered input!!!!!!: What are the services that you can offer?
filtered input!!!!!!: What are the services that you can offer?
filtered input!!!!!!: What are the services that you can offer?


2025-05-17 19:31:41,418 - INFO - HTTP Request: POST https://azopenaires.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-05-17 19:31:41,428 - INFO - OpenAI usage: CompletionUsage(completion_tokens=530, prompt_tokens=16, total_tokens=546, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-05-17 19:31:41,450 - INFO - FunctionInvoked - TestPlugin.echo
2025-05-17 19:31:41,453 - INFO - Function TestPlugin-echo succeeded.
2025-05-17 19:31:41,455 - INFO - Function completed. Duration: 3.902318s


Filtered Output: I can provide a wide range of services to help you with information, problem-solving, learning, and creativity. Here's a breakdown of the main areas I can assist with:

---

### **1. General Information & Knowledge**
- Provide explanations on a wide variety of topics (science, technology, history, arts, culture, etc.).
- Clarify complex concepts in simple terms.
- Summarize research, articles, or technical data.

---

### **2. Writing & Editing**
- Draft and edit emails, essays, reports, blog posts, or articles.
- Help with creative writing (short stories, poetry, dialogues, ideas).
- Proofread and improve grammar, structure, and tone for written content.

---

### **3. Learning & Education Support**
- Offer tutoring in subjects like math, science, literature, etc.
- Generate study guides, summaries, or flashcards.
- Explain concepts or provide test preparation strategies.
- Help you learn a new skill or subject step by step.

---

### **4. Code & Tech Assistance**
- W